In [1]:
from collections import Counter
import random

To start with, we’ll need a function to randomly choose an index based on an arbitrary set of weights:

In [2]:
def sample_from(weights):
    """returns i with probability weights[i] / sum(weights)"""

    total = sum(weights)
    rnd = total * random.random() # uniform between 0 and total
    for i, w in enumerate(weights):
        rnd -= w # return the smallest i such that
        if rnd <= 0: 
            return i # weights[0] + ... + weights[i] >= rnd

In [3]:
sample_from([1, 1, 3])

2

For instance, if you give it weights [1, 1, 3], then one-fifth of the time it will return 0, one-fifth of the time it will return 1, and three-fifths of the time it will return 2.

In [4]:
documents = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

K=4

And we’ll try to find K = 4 topics.

In order to calculate the sampling weights, we’ll need to keep track of several counts. Let’s first create the data structures for them.

How many times each topic is assigned to each document:

In [5]:
# a list of Counters, one for each document
document_topic_counts = [Counter() for _ in documents]

In [6]:
document_topic_counts

[Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter()]

How many times each word is assigned to each topic:

In [7]:
# a list of Counters, one for each topic
topic_word_counts = [Counter() for _ in range(K)]

In [8]:
# topic_word_counts

The total number of words assigned to each topic:

In [9]:
# a list of numbers, one for each topic
topic_counts = [0 for _ in range(K)]

In [10]:
topic_counts

[0, 0, 0, 0]

The total number of words contained in each document:

In [11]:
# a list of numbers, one for each document
document_lengths = list(map(len, documents))

The number of distinct words:

In [12]:
distinct_words = set(word for document in documents for word in document)
W = len(distinct_words)

And the number of documents:

In [13]:
D = len(documents)

For example, once we populate these, we can find the number of words in documents[3] associated with topic 1 as:



In [14]:
document_topic_counts[3][1]

0

And we can find the number of times nlp is associated with topic 2 as:

In [15]:
topic_word_counts[2]["nlp"]

0

Now we’re ready to define our conditional probability functions. As in Chapter 13, each has a smoothing term that ensures every topic has a nonzero chance of being chosen in any document and that every word has a nonzero chance of being chosen for any topic:

In [16]:
def p_topic_given_document(topic, d, alpha=0.1):
    """the fraction of words in document _d_
    that are assigned to _topic_ (plus some smoothing)"""
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    """the fraction of words assigned to _topic_
    that equal _word_ (plus some smoothing)"""
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + W * beta))

We’ll use these to create the weights for updating topics:

In [17]:
def topic_weight(d, word, k):
    """given a document and a word in that document,
    return the weight for the kth topic"""
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k)
                        for k in range(K)])

There are solid mathematical reasons why topic_weight is defined the way it is, but their details would lead us too far afield. Hopefully it makes at least intuitive sense that—given a word and its document—the likelihood of any topic choice depends on both how likely that topic is for the document and how likely that word is for the topic.

This is all the machinery we need. We start by assigning every word to a random topic, and populating our counters appropriately:

In [18]:
random.seed(0)
document_topics = [[random.randrange(K) for word in document]
                   for document in documents]

for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

In [19]:
document_topics

[[3, 3, 0, 2, 3, 3, 2],
 [3, 2, 1, 1, 2],
 [1, 0, 2, 1, 2, 0],
 [0, 2, 3, 0, 2],
 [3, 2, 1, 3],
 [3, 2, 0, 0, 0, 3],
 [0, 3, 2, 1],
 [2, 0, 1, 1],
 [1, 1, 3, 0],
 [0, 2, 3, 0],
 [2, 2, 0],
 [2, 1, 2, 3],
 [0, 3, 2],
 [1, 2, 1, 1, 1],
 [0, 2, 3]]

In [20]:
topic_counts

[16, 15, 20, 16]

In [21]:
word

'support vector machines'

In [22]:
document_topic_counts

[Counter({0: 1, 2: 2, 3: 4}),
 Counter({1: 2, 2: 2, 3: 1}),
 Counter({0: 2, 1: 2, 2: 2}),
 Counter({0: 2, 2: 2, 3: 1}),
 Counter({1: 1, 2: 1, 3: 2}),
 Counter({0: 3, 2: 1, 3: 2}),
 Counter({0: 1, 1: 1, 2: 1, 3: 1}),
 Counter({0: 1, 1: 2, 2: 1}),
 Counter({0: 1, 1: 2, 3: 1}),
 Counter({0: 2, 2: 1, 3: 1}),
 Counter({0: 1, 2: 2}),
 Counter({1: 1, 2: 2, 3: 1}),
 Counter({0: 1, 2: 1, 3: 1}),
 Counter({1: 4, 2: 1}),
 Counter({0: 1, 2: 1, 3: 1})]

In [23]:
topic_word_counts

[Counter({'Big Data': 1,
          'C++': 1,
          'HBase': 1,
          'Hadoop': 1,
          'Haskell': 1,
          'Java': 1,
          'R': 1,
          'artificial intelligence': 1,
          'libsvm': 1,
          'pandas': 2,
          'regression': 1,
          'scikit-learn': 2,
          'statistics': 1,
          'statsmodels': 1}),
 Counter({'Cassandra': 1,
          'HBase': 1,
          'Mahout': 1,
          'MongoDB': 1,
          'MySQL': 1,
          'Postgres': 1,
          'Python': 1,
          'databases': 1,
          'decision trees': 1,
          'deep learning': 2,
          'neural networks': 2,
          'numpy': 1,
          'theory': 1}),
 Counter({'C++': 1,
          'Cassandra': 1,
          'HBase': 1,
          'Java': 2,
          'MongoDB': 1,
          'Postgres': 1,
          'Python': 2,
          'R': 2,
          'artificial intelligence': 1,
          'machine learning': 1,
          'mathematics': 1,
          'probability': 1,
         

Our goal is to get a joint sample of the topics-words distribution and the documents-topics distribution. We do this using a form of Gibbs sampling that uses the conditional probabilities defined previously:

In [24]:
for iter in range(1000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):

            # remove this word / topic from the counts
            # so that it doesn't influence the weights
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1

            # choose a new topic based on the weights
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic

            # and now add it back to the counts
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

In [25]:
document_topics

[[0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1],
 [2, 2, 1, 1, 3, 3],
 [2, 3, 3, 2, 3],
 [1, 2, 1, 2],
 [2, 2, 0, 0, 2, 0],
 [3, 3, 2, 3],
 [1, 2, 2, 1],
 [1, 1, 0, 1],
 [0, 0, 0, 0],
 [3, 3, 3],
 [3, 0, 3, 3],
 [3, 3, 3],
 [1, 1, 1, 1, 1],
 [2, 2, 2]]

What are the topics? They’re just numbers 0, 1, 2, and 3. If we want names for them we have to do that ourselves. Let’s look at the five most heavily weighted words for each (Table 20-1):

In [28]:
for k, word_counts in enumerate(topic_word_counts):
    for word, count in word_counts.most_common():
        if count > 0: print(k, word, count)

0 Java 3
0 Big Data 3
0 Hadoop 2
0 HBase 1
0 Spark 1
0 C++ 1
0 deep learning 1
0 MapReduce 1
0 programming languages 1
0 Cassandra 1
0 Storm 1
1 HBase 2
1 MongoDB 2
1 neural networks 2
1 Postgres 2
1 machine learning 2
1 scipy 1
1 deep learning 1
1 artificial intelligence 1
1 NoSQL 1
1 MySQL 1
1 databases 1
1 Cassandra 1
1 numpy 1
1 decision trees 1
2 regression 3
2 libsvm 2
2 Python 2
2 scikit-learn 2
2 R 2
2 Haskell 1
2 mathematics 1
2 support vector machines 1
2 Mahout 1
3 statistics 3
3 probability 3
3 Python 2
3 pandas 2
3 R 2
3 statsmodels 2
3 C++ 1
3 artificial intelligence 1
3 theory 1


Based on these I’d probably assign topic names:

In [29]:
topic_names = ["Big Data and programming languages",
               "Python and statistics",
               "databases",
               "machine learning"]

at which point we can see how the model assigns topics to each user’s interests:

In [30]:
for document, topic_counts in zip(documents, document_topic_counts):
    print(document)
    for topic, count in topic_counts.most_common():
        if count > 0:
            print(topic_names[topic], count)
    print()

['Hadoop', 'Big Data', 'HBase', 'Java', 'Spark', 'Storm', 'Cassandra']
Big Data and programming languages 7

['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres']
Python and statistics 5

['Python', 'scikit-learn', 'scipy', 'numpy', 'statsmodels', 'pandas']
Python and statistics 2
databases 2
machine learning 2

['R', 'Python', 'statistics', 'regression', 'probability']
machine learning 3
databases 2

['machine learning', 'regression', 'decision trees', 'libsvm']
Python and statistics 2
databases 2

['Python', 'R', 'Java', 'C++', 'Haskell', 'programming languages']
Big Data and programming languages 3
databases 3

['statistics', 'probability', 'mathematics', 'theory']
machine learning 3
databases 1

['machine learning', 'scikit-learn', 'Mahout', 'neural networks']
Python and statistics 2
databases 2

['neural networks', 'deep learning', 'Big Data', 'artificial intelligence']
Python and statistics 3
Big Data and programming languages 1

['Hadoop', 'Java', 'MapReduce', 'Big Data']
Big D